In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D

In [3]:
def logrho0(x):
    '''
    Log likelihood for normal distribution
    '''
    
    # return np.diag(x.T@x)
    res = np.einsum('ik,ik->k', x, x)
    return res

In [ ]:
def euler_sde(x, t, f, s, h):
    '''
    One step of Euler-Mayruyama scheme
    '''
    
    if callable(f): f = f(x, t)
    if callable(s): s = s(x, t)
        
    e = np.random.randn(*x.shape)
    
    return x + h * f + np.sqrt(h) * s * e